In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error , mean_absolute_error, mean_squared_error,r2_score
from sklearn.preprocessing import OneHotEncoder
import joblib

In [22]:
data = pd.read_csv('/Users/abhishekyadav/Desktop/Python/flask/dataset/vehicle_price_prediction.csv')
data.head()

,make,model,year,mileage,engine_hp,transmission,fuel_type,drivetrain,body_type,exterior_color,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,mileage_per_year,brand_popularity,price
0,Volkswagen,Jetta,2016,183903,173,Manual,Electric,RWD,Sedan,Blue,Brown,5,NaN,Dealer,Excellent,EX,9,20433.666667,0.040054,7208.52
1,Lexus,RX,2010,236643,352,Manual,Gasoline,FWD,Sedan,Silver,Beige,5,Minor,Dealer,Good,LX,15,15776.200000,0.039921,6911.81
2,Subaru,Crosstrek,2016,103199,188,Automatic,Diesel,AWD,Sedan,Silver,Beige,5,NaN,Dealer,Excellent,Touring,9,11466.555556,0.040230,11915.63
3,Cadillac,Lyriq,2016,118889,338,Manual,Gasoline,AWD,SUV,Black,Gray,3,NaN,Private,Good,Base,9,13209.888889,0.039847,25984.79
4,Toyota,Highlander,2018,204170,196,Manual,Diesel,FWD,Sedan,Red,Brown,5,Minor,Dealer,Excellent,Sport,7,29167.142857,0.039627,8151.30


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 20 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   make              1000000 non-null  object 
 1   model             1000000 non-null  object 
 2   year              1000000 non-null  int64  
 3   mileage           1000000 non-null  int64  
 4   engine_hp         1000000 non-null  int64  
 5   transmission      1000000 non-null  object 
 6   fuel_type         1000000 non-null  object 
 7   drivetrain        1000000 non-null  object 
 8   body_type         1000000 non-null  object 
 9   exterior_color    1000000 non-null  object 
 10  interior_color    1000000 non-null  object 
 11  owner_count       1000000 non-null  int64  
 12  accident_history  249867 non-null   object 
 13  seller_type       1000000 non-null  object 
 14  condition         1000000 non-null  object 
 15  trim              1000000 non-null  object 
 16  v

In [24]:
numerical_cols = data.select_dtypes(include=['int64','float64']).columns.values
numerical_cols

array(['year', 'mileage', 'engine_hp', 'owner_count', 'vehicle_age',
       'mileage_per_year', 'brand_popularity', 'price'], dtype=object)

In [25]:
data[numerical_cols].corr()

,year,mileage,engine_hp,owner_count,vehicle_age,mileage_per_year,brand_popularity,price
year,1.000000,-0.784029,0.000135,-0.649004,-0.998898,-0.116319,-0.000099,0.664730
mileage,-0.784029,1.000000,0.000011,0.513928,0.783262,0.602885,-0.000868,-0.618349
engine_hp,0.000135,0.000011,1.000000,0.000411,-0.000143,-0.000161,0.077402,0.653339
owner_count,-0.649004,0.513928,0.000411,1.000000,0.646077,0.094390,-0.000830,-0.449437
vehicle_age,-0.998898,0.783262,-0.000143,0.646077,1.000000,0.100656,0.000234,-0.662811
mileage_per_year,-0.116319,0.602885,-0.000161,0.094390,0.100656,1.000000,-0.002629,-0.216451
brand_popularity,-0.000099,-0.000868,0.077402,-0.000830,0.000234,-0.002629,1.000000,0.055940
price,0.664730,-0.618349,0.653339,-0.449437,-0.662811,-0.216451,0.055940,1.000000


In [26]:
data_new = data.drop(columns=['year','mileage'])
data_new.head()

,make,model,engine_hp,transmission,fuel_type,drivetrain,body_type,exterior_color,interior_color,owner_count,accident_history,seller_type,condition,trim,vehicle_age,mileage_per_year,brand_popularity,price
0,Volkswagen,Jetta,173,Manual,Electric,RWD,Sedan,Blue,Brown,5,NaN,Dealer,Excellent,EX,9,20433.666667,0.040054,7208.52
1,Lexus,RX,352,Manual,Gasoline,FWD,Sedan,Silver,Beige,5,Minor,Dealer,Good,LX,15,15776.200000,0.039921,6911.81
2,Subaru,Crosstrek,188,Automatic,Diesel,AWD,Sedan,Silver,Beige,5,NaN,Dealer,Excellent,Touring,9,11466.555556,0.040230,11915.63
3,Cadillac,Lyriq,338,Manual,Gasoline,AWD,SUV,Black,Gray,3,NaN,Private,Good,Base,9,13209.888889,0.039847,25984.79
4,Toyota,Highlander,196,Manual,Diesel,FWD,Sedan,Red,Brown,5,Minor,Dealer,Excellent,Sport,7,29167.142857,0.039627,8151.30


In [27]:
#droppping redundant columns
red_cols = ['model','trim','interior_color','exterior_color','body_type']
data_new = data_new.drop(columns=red_cols)

In [28]:
data_new.head()

,make,engine_hp,transmission,fuel_type,drivetrain,owner_count,accident_history,seller_type,condition,vehicle_age,mileage_per_year,brand_popularity,price
0,Volkswagen,173,Manual,Electric,RWD,5,NaN,Dealer,Excellent,9,20433.666667,0.040054,7208.52
1,Lexus,352,Manual,Gasoline,FWD,5,Minor,Dealer,Good,15,15776.200000,0.039921,6911.81
2,Subaru,188,Automatic,Diesel,AWD,5,NaN,Dealer,Excellent,9,11466.555556,0.040230,11915.63
3,Cadillac,338,Manual,Gasoline,AWD,3,NaN,Private,Good,9,13209.888889,0.039847,25984.79
4,Toyota,196,Manual,Diesel,FWD,5,Minor,Dealer,Excellent,7,29167.142857,0.039627,8151.30


In [29]:
data_new['make'].unique()

array(['Volkswagen', 'Lexus', 'Subaru', 'Cadillac', 'Toyota',
       'Land Rover', 'Mazda', 'Ram', 'Chrysler', 'GMC', 'Volvo', 'Audi',
       'Chevrolet', 'Tesla', 'Hyundai', 'Ford', 'Porsche', 'Acura',
       'Nissan', 'Kia', 'Jeep', 'BMW', 'Dodge', 'Mercedes-Benz', 'Honda'],
      dtype=object)

In [30]:
data_new.isnull().sum()

make                     0
engine_hp                0
transmission             0
fuel_type                0
drivetrain               0
owner_count              0
accident_history    750133
seller_type              0
condition                0
vehicle_age              0
mileage_per_year         0
brand_popularity         0
price                    0
dtype: int64

In [31]:
data_new['accident_history'].unique()

array([nan, 'Minor', 'Major'], dtype=object)

In [32]:
data_new = data_new.fillna('None')
data_new['accident_history'].unique()

array(['None', 'Minor', 'Major'], dtype=object)

In [33]:
cat_cols = ['make','transmission','fuel_type','drivetrain','accident_history','seller_type','condition']

In [34]:
data_encoded = pd.get_dummies(data_new,columns=cat_cols,drop_first=False).astype(int)
data_encoded.head()

,engine_hp,owner_count,vehicle_age,mileage_per_year,brand_popularity,price,make_Acura,make_Audi,make_BMW,make_Cadillac,...,drivetrain_FWD,drivetrain_RWD,accident_history_Major,accident_history_Minor,accident_history_None,seller_type_Dealer,seller_type_Private,condition_Excellent,condition_Fair,condition_Good
0,173,5,9,20433,0,7208,0,0,0,0,...,0,1,0,0,1,1,0,1,0,0
1,352,5,15,15776,0,6911,0,0,0,0,...,1,0,0,1,0,1,0,0,0,1
2,188,5,9,11466,0,11915,0,0,0,0,...,0,0,0,0,1,1,0,1,0,0
3,338,3,9,13209,0,25984,0,0,0,1,...,0,0,0,0,1,0,1,0,0,1
4,196,5,7,29167,0,8151,0,0,0,0,...,1,0,0,1,0,1,0,1,0,0


In [35]:
X = data_encoded.drop('price',axis=1)
y = data_encoded['price']

In [36]:
X_train , X_test , y_train ,y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [37]:
model = LinearRegression()
model.fit(X_train,y_train)

LinearRegression()

In [38]:
y_pred = model.predict(X_test)

/Users/abhishekyadav/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: divide by zero encountered in matmul
  return X @ coef_ + self.intercept_
/Users/abhishekyadav/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: overflow encountered in matmul
  return X @ coef_ + self.intercept_
/Users/abhishekyadav/Library/Python/3.9/lib/python/site-packages/sklearn/linear_model/_base.py:279: RuntimeWarning: invalid value encountered in matmul
  return X @ coef_ + self.intercept_


In [39]:
mae = mean_absolute_error(y_pred,y_test)
mse = mean_squared_error(y_pred,y_test)
rmse = root_mean_squared_error(y_pred,y_test)
r2 = r2_score(y_pred,y_test) #provides a measure of how well the observed outcomes are replicated by the model
print('mean absolute error is',mae)
print('mean absolute error is',mse)
print('mean absolute error is',rmse)
print('mean absolute error is',r2)

mean absolute error is 2424.126791923456
mean absolute error is 12330183.623504488
mean absolute error is 3511.436119809741
mean absolute error is 0.9291411304162422


In [40]:
joblib.dump(model, "regression_model.pkl")

['regression_model.pkl']